# Data Reply - QMU ML Challenge
## Introduction

In this challenge, you will be using US census data to predict whether or not a person earns over $50k a year.
This challenge takes you through the data science process of Data Loading, Data Exploration & Cleansing, Feature Generation, Model Calibration and Model Evaluation. You will build different models to tackle this classification problem, evaluating your models along the way, and finally combining each of your models into your own ensemble classifier.

---

### The DS process

There are 5 main steps in the data science process. 

1. Data loading. 
2. Data exploration and clensing. 
3. Feature generation.
4. Model calibration.
5. Model evaluation.

The majority of a data scientists time is spend on number 2. Exploration and clensing are a very important step in the data science process with data scientists typically spending 80% of their time preparing the data.

We will be following this process to demonstrate how ML projects are developped. 

---

### 1. Data Loading

In this example, we are using a relatively small and static dataset - so the data loading process is straight forward. 

In real-life projects, the data:
 - is sometimes dynamic - updated regularly, or streaming into the data storage system in real time
 - is often very large - 100s of GBs of TBs
 - is ALWAYS dirty, and needs cleaning

In these cases, some specialised big data or streaming tools may be needed to develop the solution, such as spark, kafke, etc. But, for this example, we will consider this static dataset.

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from zipfile import ZipFile

*Read the data into a dataframe using pandas, passing the argument names = [...] into the read_csv function to set the column names.*

In [ ]:
zip_file = ZipFile('census_data.zip')
df = pd.read_csv(zip_file.open('census_data.csv'))

*We can inspect a sample of the dataframe by using the df.head( ) function.*

In [ ]:
df.head(n=5)

*You will notice that some of the columns are obscured from view. This is a default setting and can be overwritten using the following command.*

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
df.head(n=5)

---
### 2. Data Exploration & Clensing

*Now that the data is loaded into a dataframe, we can look at some summary statistics of numerical columns to give us a first idea of the data.*

In [ ]:
df.describe()

---
*Different features of the data may need different preprocessing steps. For example, "sex" might be encoded to 0 and 1 to represent male and female\*, but doing this with "wage_per_hour" - a continuous data type - wouldn't make sense.*

\**NOTE: the US census dataset used here uses biological "sex", for which there were 2 options (male and female), as opposed to gender where may be more.*

*There are 4 different 'types' of feature which may be present in the data: nominal, ordinal, interval and ratio. Each of these should be dealt with in different ways. [This website](https://www.graphpad.com/support/faq/what-is-the-difference-between-ordinal-interval-and-ratio-variables-why-should-i-care/) gives some more context on these different feature types.*

*From these definitions, we can see that "sex" is obviously a nominal feature. If there are other nominal features in the dataset, it might be useful to create some kind of function which can be applied to each nominal column - then also for each ordinal, interval and ratio columns. To do this we shall make use of sklearn's transformers - more on these later.*

*But first, we need to determine which features belong to each of these categories.*

*We can inspect the columns and their datatypes by running df.dtypes in the next cell.*

In [ ]:
df.dtypes

---
As we can see, our features are either int64, or object datatypes. (Here, object can be thought of as string.)

Inspecting columns of each datatype in turn might be a good place to start when determining which columns are nominal, etc. 
Remember though, ALL DATA IS DIRTY. 
Columns may need to be changed from one datatype to another, this is just a starting point.


Complete the function in the next cell to return a list of all columns from a dataframe of a given datatype.

In [ ]:
def get_cols_of_type(input_df, data_type):
    '''
    Takes a dataframe and dtype, eg. string, and returns a list of columns names of that dtype.
    '''
    output_list = []
    ### fill in logic here.
    ### fill in logic here.
    ### fill in logic here.    
    
    return output_list


In [ ]:
# display int datatypes from our dataframe
get_cols_of_type(df, int)

In [ ]:
# display object datatypes from our dataframe
get_cols_of_type(df, object)

---
*Now let's go through each column and decide which category each belongs to. Then we can deal with each feature type in turn by passing our list of columns into our transformer function.*

**Remember** - earn_over_50k will be our target variable, so we want to keep that separate.

In [ ]:
# Go through the features in the data and fill in the lists of columns for each feature type.

nominal_cols = []

ordinal_cols = []

interval_cols = []

ratio_cols = []

target_col = ["earn_over_50k"]

**Hint**: something which might be useful to inspect what values a feature can take is df.column.unique( )

---
*Once you have finished, perform a quick check that the number of columns in the dataframe matches the number of elements in all your lists combined, using 'assert'.*

In [ ]:
assert len(nominal_cols+ordinal_cols+interval_cols+ratio_cols+target_col) == len(df.columns)

---
*Now let's look at each feature type in turn and see how we will deal with them, starting with nominal features. 
The following for look allows us to inspect all the possbile values which each nominal feature can take. *


In [ ]:
for feature in nominal_cols:
    print(feature+":")
    print(sorted(df[feature].unique()))
    print("")

*One of the first things to notice is that for the country_of_birth_X columns, we have some missing values represented by "?". Dealing with missing values is a common thing to have to do in any data science project. Some typical methods for dealing with missing numerical values include mean imputation and median imputation. *

**Think of a way to impute the missing values in this case for the categorical columns, country_of_birth_X. ([Hint](https://pandas.pydata.org/pandas-docs/stable/reference/series.html))**

In [ ]:
for feature in ["country_of_birth_father", "country_of_birth_mother", "country_of_birth_self"]:
    ### fill in the following line with some logic to replace "?" by the modal country of birth
    ...

*Now if we look again at the values these three features can take, we should see that "?" is no longer an option.*

In [ ]:
for feature in nominal_cols:
    print(feature+":")
    print(sorted(df[feature].unique()))
    print("")

*Notice that some form of "Not in universe" comes up a lot for missing values in the different fields. When we use transformers later on, it would be useful to be able to handle all of these in the same way.*

**So as another preprocessing step, change all instances containing "Not in universe" to "null".**

In [ ]:
df = df.replace("Not in universe", "null").replace("Not in universe or children", "null")\
                                          .replace("Not in universe under 1 year old", "null")

In [ ]:
for feature in nominal_cols:
    print(feature+":")
    print(sorted(df[feature].unique()))
    print("")

---

### 3. Feature generation
*Now that we see all the possible values which each nominal feature can take, we need to decide on how to handle them.*

*For some features, we may wish to convert them into binary variables, such as own_business_or_self_employed, yes or not yes.*

*For others, such as for each country_of_birth_{}, we might wish to binarize these to something like from "United-States", yes or no?*

*Other features should not be binarized, such as marital_status, and we will have to use [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).*

**Come up with a strategy to encode the following columns as binary variables:**
\[ sex, live_in_this_house_1_year_ago,  country_of_birth_father,  country_of_birth_mother,  country_of_birth_self,  own_business_or_self_employed \]

In [ ]:
df["is_male"] = np.where(df['sex']=="Male", 1, 0)
df["is_lived_in_this_house_1_year_ago"]= ### Fill in logic
df["born_usa_father"] = ### Fill in logic
df["born_usa_mother"] = ### Fill in logic
df["born_usa_self"]   = ### Fill in logic
df["is_self_employed"]= ### Fill in logic


*Now drop the columns we no longer need.*

In [ ]:
drop_cols = ["sex", "live_in_this_house_1_year_ago", "country_of_birth_father",
            "country_of_birth_mother", "country_of_birth_self", "own_business_or_self_employed"]
df = df.drop(columns = drop_cols)

In [ ]:
df.head()

*Now we have manually dealt with some of the nominal_cols, we can disguard them from the list of nominal columns we were dealing with. To do this, we make use of sets.*

*The remaining nominal_cols will need to be one-hot encoded.*

In [ ]:
remaining_nominal_cols = list(set(nominal_cols)-set(drop_cols))

In [ ]:
remaining_nominal_cols

---
##### Ordinal features
*To deal with ordinal values, we need to do a little more.
First we need to create an order and map that order to increasing integers which represent that order. Let's take a look at the values education can take.*

In [ ]:
sorted(df.education.unique())

*Clearly there is some kind of order to this. Completing 10th grade is better than completing 9th grade, having a phd is better than having a masters, etc.*

*Let's try and put some kind of order to the elements in that list in the form of a dictionary, with the least advanced education starting at 0, and the most advanced education being the greatest number.*

*The benefit of the dictionary is that different keys can have the same value, thus if you think 2 different educations are equal, you can assign them the same value.*

In [ ]:
### Complete the dictionary to reasonably encapsulate the order in the data
education_dict = {'Children':0,
                 'Less than 1st grade':1 
                 }

*Now we can use the [pandas.Series.map](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) function to map these categorical values to numerically ordered ones.*

In [ ]:
df["education"] = df["education"].map(education_dict)

In [ ]:
df.head()

---
##### Ratio features
*We will, however, be considering the ratio columns. Given that the ratio columns have different scales which can skew the model massively in favour of a particular feature, we should apply some technique to mitigate this. 
A typical technique to use is to scale each of the features so that they are on the same scale, usualy the interval [0,1].
To do this we can use min-max scaler in sklearn.*

##### Target feature

*For the rest of the notebook and analysis, we shall keep the target variable separate from the other variables. We will also transform this variable to a binary varable with 1 representing those earning over 50k.*

In [ ]:
target = np.where(df["earn_over_50k"]=="-50000", 0, 1)

##### Combining all the features together with transformers

In [ ]:
df.head()

In [ ]:
sorted(df.columns)

In [ ]:
# these are the columns we binarized earlier with custom conditions
bin_cols=["education", "is_male", "is_lived_in_this_house_1_year_ago", "born_usa_father",
                      "born_usa_mother", "born_usa_self", "is_self_employed"]

cols = ratio_cols+bin_cols+remaining_nominal_cols

data = df[cols]

data.head()

*Now we can see the dataframe with numeric features to the left, and our nominal features to the right.
The next step is to utilise the Scikit learn's ColumnTransformer and one-hot encoder to finish the pre-processing of the data frame.*

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ratio_cols+bin_cols

numeric_transformer = Pipeline(steps=[
                        ('imputer', SimpleImputer(strategy='median')),
                        ('scaler', MinMaxScaler())])

categorical_features = remaining_nominal_cols

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='null')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [ ]:
processed_data = preprocessor.fit_transform(data)

In [ ]:
processed_data

In [ ]:
data.shape

*Notice 2 things:*
1. *the number of features after proprocessing has increased substantially from 22 to 92 - this is because of the one hot encoding creating binary variables for each possible category.*
2. *the data is now stored as a sparse matrix. Sparse matrices are a more efficient way of storing data which is mostly 0s but contains a few non-zero values.*
---

### A quick note on overfitting
*In this section, we will illustrate the notion of overfitting. To do this we will first generate some synthetic data.*

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
from IPython import display
from sklearn.externals.six import StringIO
import numpy as np
from matplotlib import pylab as plt
 
# Create a random dataset
rng = np.random.RandomState(42) # fix the seed so the result are stable
N_points = 400
X = np.sort(5 * rng.rand(N_points, 1), axis=0)
y = np.sin(X).ravel() + .4 * (0.5 - rng.rand(N_points))

plt.plot(X, y, 'b.')
plt.xlabel('Data')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

*So, above is our example dataset and suppose we want fit a decision tree regressor to estimate the value of unseen data. (You don't need to know anything about decision trees at this stage. But if you want to find out more, you can read more about decision trees [here](http://scikit-learn.org/stable/modules/tree.html).) All you need to know is that max_depth is a hyperparameter of the decision tree and if we increase that hyperparameter, the tree can model more complexity in the data.*

In [ ]:
from ipywidgets import interactive, IntSlider
from sklearn import tree

def train_and_plot(max_depth):
    est = tree.DecisionTreeRegressor(max_depth=max_depth)
    est.fit(X, y)

    plt.plot(X, y, 'b.', label='data')
    line = plt.plot(X, est.predict(X), 'r-', label='model')
    plt.setp(line, linewidth=3.)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend(loc='upper right');
    plt.show()
    
max_depth_slider=IntSlider(min=1,max=8,step=1,value=2)
interactive(train_and_plot, max_depth=max_depth_slider)

**Exercise:** Try playing around with `max_depth` in the above above cell. What happens when you set it to `max_depth = 2`.  Does the fitted model look better or worse?  What about `4` or `8`?  We can tell (visually) that `max_depth = 2` is *underfitting* and `max_depth = 8` is *overfitting*. But how can we do this computationally?

#### In-sample (Training) error
*Naively, you might think that we could just measure the error of the model and choose the model with the best error.  For example, let's define the error as the mean squared error (MSE).  Let's try that below:*

In [ ]:
from sklearn import metrics

max_depths = range(1, 10)
in_sample_errors = []
for max_depth in max_depths:
    y_pred = tree.DecisionTreeRegressor(max_depth=max_depth).fit(X, y).predict(X)
    in_sample_errors.append(metrics.mean_squared_error(y, y_pred))
    
plt.plot(max_depths, in_sample_errors, label='In-Sample Error')
plt.xlabel('max_depth')
plt.ylabel('MSE')
plt.legend(loc='upper right');

*We can see that the error is decreasing with the depth of the tree. Our visual inspection told us models with max_depth = 8 completely over-fit the error. It turns out that the In-Sample Error that we calculated above will always decrease with the complexity of the model (in this case, the depth of the tree). We can see from above that this leads us to overfit the data. In order to test how well our model generalizes, we need to see how it performs on new data.*

#### Out of sample (Testing) error

*One way to do this is to (randomly) split the data into training and test sets.  We train on the training set and test the resulting model on the test set.  Since the trained model never saw the test data, we can evaluate the performance on the test data.*

In [ ]:
from sklearn import model_selection
from sklearn import metrics

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
test_errors = []
for max_depth in max_depths:
    est = tree.DecisionTreeRegressor(max_depth=max_depth).fit(X_train, y_train)
    y_pred = est.predict(X_test)
    test_errors.append(metrics.mean_squared_error(y_test, y_pred))

plt.plot(max_depths, in_sample_errors, label='In-Sample Error')
plt.plot(max_depths, test_errors, label='Out-of-Sample Error')
plt.xlabel('max_depth')
plt.ylabel('MSE')
plt.legend(loc='upper right');

This is the picture we were looking for!  The *In-Sample Error (Training)* is always decreasing but the out *Out-of-Sample Error (Testing)* initially decreases with model complexity (higher `max_depth`) but ultimately increases again.  The *Out-of-Sample Error (Testing)* is the metric to look at when evaluating overfitting.

It turns out that this is an illustration of a very general problem in machine-learning called **Bias-Variance tradeoff** (the concept is so general that it even has a [Wikipedia article](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_dilemma).  A more rigorous account can be found [here](http://www.brnt.eu/phd/node14.html)).  The tradeoff tells us that we can decompose our *Out-of-Sample Error* into

$$ \mbox{Out-of-Sample Error} = \mbox{Bias} + \mbox{Variance}. $$

The *Bias* corresponds to how far off we expect the model to deviate from reality (i.e. the model's bias) because of parametric assumptions (e.g. we forced the model to be linear or to be a tree of maximum depth 2).  It is given by the *In-Sample Error* of the above plot and always goes down with complexity.  High Bias models correspond to *underfitting*.

The *Variance* accounts for the fact that the model was only trained on a (noisy) subset of the data and that the idiosyncratic noise in the data is therefore likely to contribute some variance to the model.  The more complex we allow the model to be, the more likely we are to overfit by picking up more of this noise.  High variance models correspond to *overfitting*.

We can also think of bias as unmodeled data and variance as modeled noise.  As we increase the complexity of the model, we will necessarily model more of the data (reduce bias, reduce underfitting) but also start modeling noise (increase variance, increase overfitting).  Here's a helpful diagram of the decomposition.  Notice that at the optimal point, we have not yet learned on all our signal (still unmodeled data left) and we have picked up some noise and overfitting.

---
### 4. and 5. - Model Calibration and Evaluation

Back to the problem at hand and it's over to you... 

Split the preprocessed data and target values into a training and test data set using train_test_split. 
Then train the model of your choice and evaluate the outcome. 


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
### import the classifier of your choice.

### complete the pipeline with classifier
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', ... )])

### complete the splitting of your data
X_train, X_test, y_train, y_test = train_test_split(...)

### evaluate your classifier
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))



**Questions:**
 - How accurate is the model? 
 - Is accuracy the best metric? What others can you think of? ([Hint](https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c))
 - What is the baseline accuracy that your model should be compared against?
 - How might you deal with unbalanced classes in your classification problem? ([Hint](https://elitedatascience.com/imbalanced-classes))
 - Can you add a grid search to improve your model performance without overfitting? ([Hint](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html#using-the-prediction-pipeline-in-a-grid-search))
 - How do other models compare to the one you made?
 - What is the best model you managed to come up with?

**Challenge:**

 - Create your own ensemble model by combining the predictions from the previous models you have built.